## DeepSeek R1 - Loan Approval
Thorough analysis of a loan aproval request with Langchain orchestrator

### Environment Setup

In [1]:
# Importing required packages
import os
from azure.identity import DefaultAzureCredential
from langchain_azure_ai.chat_models import AzureAIChatCompletionsModel
from langchain.prompts import PromptTemplate

In [2]:
# Defining model's endpoint and Azure credentials
DS_Model = os.getenv("AZURE_FOUNDRY_DEEPSEEK")
DS_Region = "eastus"
DS_Endpoint = f"https://{DS_Model}.{DS_Region}.models.ai.azure.com"

Az_Credential = DefaultAzureCredential()

### DeepSeek R1 Model

In [3]:
# Initiating Azure AI Foundry client
model = AzureAIChatCompletionsModel(
    endpoint = DS_Endpoint,
    credential = Az_Credential,
    max_tokens = 2048
)

### Loan Approval Chain

In [4]:
# Prompt template
loan_approval_prompt = PromptTemplate(
    input_variables = ["applicant_info"],
    template = """
    You are a financial advisor at a bank.  You need to decide whether to approve a loan application.
    Consider the following applicant information:

    {applicant_info}

    Provide your decision (approve or deny) AND a detailed explanation of your reasoning, including specific factors you considered and how they influenced your decision.  Be transparent and thorough, as this is for a regulated industry.
    """
)

In [5]:
# Defining the chain
chain = loan_approval_prompt | model

### Long-response Completion

In [6]:
# Sample applicant
applicant_info = """
Applicant Name: John Doe
Credit Score: 680
Loan Amount Requested: $10,000
Annual Income: $60,000
Employment History: Stable employment for 5 years
Debt-to-Income Ratio: 0.35
Loan Purpose: Home Renovation
"""

In [7]:
# Getting decision and explanation
response = chain.invoke(applicant_info)

print("Response:", response.content)
print("----------------------------------------")
print("Usage:")
print("\tPrompt tokens:", response.usage_metadata["input_tokens"])
print("\tCompletion tokens:", response.usage_metadata["output_tokens"])
print("\tTotal tokens:", response.usage_metadata["total_tokens"])

Response: <think>
Okay, so I need to decide whether to approve John Doe's loan application. Let me start by looking at his details carefully.

His credit score is 680. I know credit scores are categorized, and 680 is just below the "good" threshold, which is usually around 700. But different lenders have different standards. Maybe this bank considers 680 as acceptable with other compensating factors. 

He wants a $10,000 loan for home renovation. His annual income is $60k, which breaks down to about $5k a month. Debt-to-income ratio is 0.35, so 35%. That means his monthly debt payments are 35% of his income. Wait, the DTI here might be calculated as total monthly debts over monthly income. If his income is $5k/month, 35% DTI is $1,750 in monthly debt obligations. But is the new loan payment going to be added to that? Since the DTI is already 0.35, we need to check if adding the new loan won't push it over the bank's limit. Let's think: typical thresholds for DTI are around 36-43% for a

### Streaming Completion

In [8]:
# Streaming the chain - first 1000 words only
response = chain.stream(applicant_info)

max_words_to_display = 1000
words_displayed = 0

for chunk in response:
    words_in_chunk = len(chunk.content.split())
    if words_displayed + words_in_chunk > max_words_to_display:
        print("\n... (Stopping stream after 1000 words)")
        break
    
    print(chunk.content, end="", flush=True)
    words_displayed += words_in_chunk

<think>
Okay, so I'm trying to decide whether to approve or deny John Doe's loan application. Let's start by going through all the information given. His credit score is 680, which I know is near the lower end of what's considered "fair." Sometimes lenders look for a higher score, maybe 700 or above, for the best rates, but 680 isn't terrible. The loan amount he's asking for is $10,000. His annual income is $60k. Let me calculate his monthly income, which would be $60,000 divided by 12, so $5,000 per month.

The employment history is stable—he's been at his job for five years. That's a good sign because it shows consistency. Debt-to-income ratio is 0.35, which is 35%. I remember that lenders usually prefer a DTI below 36%, so this is right on the edge. If it goes over, that might be a red flag, but 35% is acceptable, though a bit high.

The purpose of the loan is home renovation. That's generally a positive reason because it can add value to the property, which is collateral if it's a 